### Importing Required Libraries

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import nltk
from nltk.stem import WordNetLemmatizer
from gensim.models import Word2Vec

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re # for regex
# ! pip install vaex

# nltk.download('all')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize,sent_tokenize
from nltk.stem import SnowballStemmer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB,MultinomialNB,BernoulliNB

from sklearn.model_selection import GridSearchCV

from sklearn.metrics import accuracy_score
import pickle
import string
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
# import vaex

from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
# from google.colab import files

import nltk


import warnings
warnings.simplefilter('ignore')

C:\Users\User\anaconda3\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


### Reading File

In [120]:
df=pd.read_csv(r'C:/data_sci_dec/internship_projects/cardekho.csv')
df.head(3)

,Unnamed: 0,Car_name,Price,Kilometers,Year of Purchase,Owner,Fuel,Transmission,RTO,Insurance Type,Feature,Safety,Interior,Exterior
0,0,2011 Maruti Ritz,"₹ 1,98,410","54,004 Kms",2010,2nd Owner,Petrol,Manual,GJ01,Third Party insurance,"Foldable Rear Seat, Power Steering, Power Wind...","Centeral Locking, Power Door Locks, Child Safe...","Air Conditioner, Heater, Adjustable Steering, ...","Adjustable Head Lights, Fog Lights Front, Fog ..."
1,1,2014 Hyundai Verna,"₹ 4,85,492","65,484 Kms",2014,2nd Owner,Diesel,Manual,GJ01,Third Party insurance,"Power Steering, Power Windows Front, Power Win...","Anti Lock Braking System, Centeral Locking, Po...","Air Conditioner, Heater, Adjustable Steering, ...","Adjustable Head Lights, Fog Lights Front, Powe..."
2,2,2018 Maruti Wagon R,"₹ 4,41,025","23,197 Kms",2018,1st Owner,Petrol,Manual,GJ18,Third Party insurance,"Foldable Rear Seat, Power Steering, Power Wind...","Centeral Locking, Power Door Locks, Child Safe...","Air Conditioner, Heater, Adjustable Steering, ...","Adjustable Head Lights, Fog Lights Front, Powe..."


In [121]:
price=[]
for i in df['Price'].str.strip("₹ "):
    price.append(int(''.join(i.split(","))))
df['Price new']=price

In [122]:
fuel=pd.get_dummies(df['Fuel'])

In [123]:
df=pd.concat([fuel,df],axis=1)

In [124]:
df.drop('Unnamed: 0',axis=1,inplace=True)

In [125]:
df['Kilometers'].str.strip('Kms')

0      54,004 
1      65,484 
2      23,197 
3      43,031 
4      24,230 
        ...   
826     5,663 
827    29,433 
828    43,155 
829    10,265 
830    44,979 
Name: Kilometers, Length: 831, dtype: object

In [126]:
kms=[]
for i in df['Kilometers'].str.strip("Kms"):
    kms.append(int(''.join(i.split(","))))
df['Kilometers new']=kms

In [127]:
df.drop(["Price","Kilometers"],axis=1,inplace=True)

In [128]:
list(df['Owner'].value_counts().keys())

['1st Owner', '2nd Owner', '3rd Owner']

In [129]:
df['Owner'].replace(list(df['Owner'].value_counts().keys()),[1,2,3],inplace=True)

In [130]:
df.drop('Fuel',axis=1,inplace=True)

In [131]:
df['RTO'].value_counts()

HR26      52
MH12      50
HR51      49
RJ14      37
KA05      29
          ..
1 2022     1
GJ08       1
MH20       1
KA52       1
RJ51       1
Name: RTO, Length: 115, dtype: int64

In [132]:
df.iloc[829,:]

CNG                                                                 0
Diesel                                                              0
LPG                                                                 0
Petrol                                                              1
Car_name                                             2018 Maruti Ciaz
Year of Purchase                                                 2018
Owner                                                               1
Transmission                                                   Manual
RTO                                                              MH12
Insurance Type                                  Third Party insurance
Feature             Power Steering, Power Windows Front, Power Win...
Safety              Anti Lock Braking System, Centeral Locking, Po...
Interior            Air Conditioner, Heater, Adjustable Steering, ...
Exterior            Adjustable Head Lights, Fog Lights Front, Powe...
Price new           

In [133]:
df.tail()

,CNG,Diesel,LPG,Petrol,Car_name,Year of Purchase,Owner,Transmission,RTO,Insurance Type,Feature,Safety,Interior,Exterior,Price new,Kilometers new
826,0,0,0,1,2018 Hyundai i20,2018,1,Manual,MH12,Third Party insurance,"Power Steering, Power Windows Front, Low Fuel ...","Anti Lock Braking System, Centeral Locking, Ch...","Air Conditioner, Heater, Digital Odometer, Tac...","Adjustable Head Lights, Manually Adjustable Ex...",565000,5663
827,0,0,0,1,2018 Maruti Alto 800,2018,1,Manual,MH12,Third Party insurance,"Power Steering, Power Windows Front, Remote Tr...","Centeral Locking, Child Safety Locks, Passenge...","Air Conditioner, Heater, Adjustable Steering, ...","Adjustable Head Lights, Manually Adjustable Ex...",375000,29433
828,0,0,0,1,2015 Honda City,2015,1,Automatic,MH12,Third Party insurance,"Power Steering, Power Windows Front, Power Win...","Anti Lock Braking System, Centeral Locking, Po...","Air Conditioner, Heater, Adjustable Steering, ...","Adjustable Head Lights, Fog Lights Front, Powe...",780056,43155
829,0,0,0,1,2018 Maruti Ciaz,2018,1,Manual,MH12,Third Party insurance,"Power Steering, Power Windows Front, Power Win...","Anti Lock Braking System, Centeral Locking, Po...","Air Conditioner, Heater, Adjustable Steering, ...","Adjustable Head Lights, Fog Lights Front, Powe...",820000,10265
830,0,0,0,1,2012 Maruti Alto K10,2012,1,MH12,6 2021,Manual,"Power Steering, Power Windows Front, Remote Tr...","Centeral Locking, Passenger Side Rear View Mir...","Air Conditioner, Heater, Tachometer, Electroni...","Adjustable Head Lights, Fog Lights Front, Manu...",225752,44979


In [134]:
df['Transmission'].value_counts()

Manual       666
Automatic    132
MH01           4
DL3C           3
UP14           2
KA01           2
HR87           2
MH03           2
KA03           2
MH12           2
MH11           2
DL4C           2
KA50           1
KA02           1
MH14           1
KA05           1
MH13           1
MH02           1
MH04           1
DL9C           1
MH46           1
KA51           1
Name: Transmission, dtype: int64

In [135]:
df['Insurance Type'].value_counts()

Third Party insurance    760
Not Available             36
Manual                    26
Automatic                  7
Third Party                2
Name: Insurance Type, dtype: int64

In [136]:
df.loc[df[df['Insurance Type'].isin(['Manual'])].index,'RTO']=df.loc[df[df['Insurance Type'].isin(['Manual'])].index,'Transmission']

In [137]:
df.loc[df[df['Insurance Type'].isin(['Automatic'])].index,'RTO']=df.loc[df[df['Insurance Type'].isin(['Automatic'])].index,'Transmission']

In [138]:
df.loc[df[df['Insurance Type'].isin(['Manual'])].index,'Transmission']="Manual"
df.loc[df[df['Insurance Type'].isin(['Automatic'])].index,'Transmission']="Automatic"

In [139]:
df.loc[df[df['Insurance Type'].isin(['Manual'])].index,'Insurance Type']="Not Available"
df.loc[df[df['Insurance Type'].isin(['Automatic'])].index,'Insurance Type']="Not Available"

In [140]:
df.loc[df[df['Insurance Type'].isin(['Third Party'])].index,'Insurance Type']="Third Party insurance"

In [141]:
df['Insurance Type'].value_counts()

Third Party insurance    762
Not Available             69
Name: Insurance Type, dtype: int64

In [142]:
df['RTO'].value_counts()

MH12    52
HR26    52
HR51    49
RJ14    37
KA03    31
        ..
GJ08     1
MH20     1
KA52     1
GJ11     1
RJ51     1
Name: RTO, Length: 102, dtype: int64

In [143]:
insusrance=pd.get_dummies(df['Insurance Type'])
transmission=pd.get_dummies(df['Transmission'])

In [144]:
df=pd.concat([insusrance,transmission,df],axis=1)
df.drop(['Transmission','Insurance Type'],axis=1,inplace=True)

In [145]:
df

,Not Available,Third Party insurance,Automatic,Manual,CNG,Diesel,LPG,Petrol,Car_name,Year of Purchase,Owner,RTO,Feature,Safety,Interior,Exterior,Price new,Kilometers new
0,0,1,0,1,0,0,0,1,2011 Maruti Ritz,2010,2,GJ01,"Foldable Rear Seat, Power Steering, Power Wind...","Centeral Locking, Power Door Locks, Child Safe...","Air Conditioner, Heater, Adjustable Steering, ...","Adjustable Head Lights, Fog Lights Front, Fog ...",198410,54004
1,0,1,0,1,0,1,0,0,2014 Hyundai Verna,2014,2,GJ01,"Power Steering, Power Windows Front, Power Win...","Anti Lock Braking System, Centeral Locking, Po...","Air Conditioner, Heater, Adjustable Steering, ...","Adjustable Head Lights, Fog Lights Front, Powe...",485492,65484
2,0,1,0,1,0,0,0,1,2018 Maruti Wagon R,2018,1,GJ18,"Foldable Rear Seat, Power Steering, Power Wind...","Centeral Locking, Power Door Locks, Child Safe...","Air Conditioner, Heater, Adjustable Steering, ...","Adjustable Head Lights, Fog Lights Front, Powe...",441025,23197
3,0,1,0,1,0,0,0,1,2017 Maruti Wagon R,2017,1,GJ18,"Foldable Rear Seat, Power Steering, Power Wind...","Centeral Locking, Power Door Locks, Child Safe...","Air Conditioner, Heater, Adjustable Steering, ...","Adjustable Head Lights, Fog Lights Front, Powe...",405188,43031
4,0,1,0,1,0,0,0,1,2019 Maruti S-Presso,2019,1,GJ01,"Power Steering, Power Windows Front, Low Fuel ...","Anti Lock Braking System, Centeral Locking, Ch...","Air Conditioner, Heater, Digital Odometer, Ele...","Adjustable Head Lights, Manually Adjustable Ex...",415500,24230
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
826,0,1,0,1,0,0,0,1,2018 Hyundai i20,2018,1,MH12,"Power Steering, Power Windows Front, Low Fuel ...","Anti Lock Braking System, Centeral Locking, Ch...","Air Conditioner, Heater, Digital Odometer, Tac...","Adjustable Head Lights, Manually Adjustable Ex...",565000,5663
827,0,1,0,1,0,0,0,1,2018 Maruti Alto 800,2018,1,MH12,"Power Steering, Power Windows Front, Remote Tr...","Centeral Locking, Child Safety Locks, Passenge...","Air Conditioner, Heater, Adjustable Steering, ...","Adjustable Head Lights, Manually Adjustable Ex...",375000,29433
828,0,1,1,0,0,0,0,1,2015 Honda City,2015,1,MH12,"Power Steering, Power Windows Front, Power Win...","Anti Lock Braking System, Centeral Locking, Po...","Air Conditioner, Heater, Adjustable Steering, ...","Adjustable Head Lights, Fog Lights Front, Powe...",780056,43155
829,0,1,0,1,0,0,0,1,2018 Maruti Ciaz,2018,1,MH12,"Power Steering, Power Windows Front, Power Win...","Anti Lock Braking System, Centeral Locking, Po...","Air Conditioner, Heater, Adjustable Steering, ...","Adjustable Head Lights, Fog Lights Front, Powe...",820000,10265


In [146]:
df.select_dtypes('O').columns

Index(['Car_name', 'RTO', 'Feature', 'Safety', 'Interior', 'Exterior'], dtype='object')

In [147]:
df['all_text']=df['Car_name']+df['RTO']+df['Feature']+df['Safety']+df['Interior']+df['Exterior']

In [148]:
cv=CountVectorizer()
text_feature=cv.fit_transform(df['all_text'])

In [149]:
text_feature.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [150]:
dff=pd.DataFrame(text_feature.toarray())

In [151]:
dff.shape

(831, 811)

In [152]:
df.select_dtypes('int64')

,Year of Purchase,Owner,Price new,Kilometers new
0,2010,2,198410,54004
1,2014,2,485492,65484
2,2018,1,441025,23197
3,2017,1,405188,43031
4,2019,1,415500,24230
...,...,...,...,...
826,2018,1,565000,5663
827,2018,1,375000,29433
828,2015,1,780056,43155
829,2018,1,820000,10265


In [153]:
df.select_dtypes('uint8')
df_new=pd.concat([dff,df.select_dtypes('uint8'),df.select_dtypes('int64')],axis=1)

In [2]:
from sklearn.naive_bayes import GaussianNB,MultinomialNB
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.linear_model import SGDRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error, mean_absolute_error,accuracy_score,r2_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline


parameters_sgd={
 'penalty': ['l1','l2','elasticnet'],
 'alpha': [0.0001,0.001,0.01,0.1,1],
    'max_iter':[1000,2000],
#     'random_state': list(np.arange(30,250)),
}
parameters_rndf={
 'criterion': ['mse','mae'],
 'max_features': ['auto','sqrt'],
 'n_estimators': list(np.arange(100,150)),
#  'random_state': list(np.arange(30,250)),
    'max_depth':[5],
}
parameters_deci_tree={
 'criterion': ['mse','mae'],
#  'splitter':['best','random'],
 'max_features': ['auto','sqrt'],
 'max_depth':list(np.arange(5,30)),
#  'random_state': list(np.arange(30,250)),
}

parameters_knn_reg={
    'n_neighbors': list(np.arange(5,10)),
    'weights':['uniform','distance'],
    
}
parameters_svr_reg={
    'kernel': ['linear','poly','rbf'],
    'gamma':['auto','scale'],
    'max_iter':[500,700],
    
}
parameters_linear_reg = {'fit_intercept':[True,False], 'normalize':[True,False], 'copy_X':[True, False]}



def best_rn_state(model,X,target):
    # for best random state
    best_state=0
    accu=0
    for i in range(30,250):
        X_train, X_test, y_train, y_test = train_test_split(X,target, test_size=0.25,random_state=i)
        md = model
        md.fit(X_train,y_train)
        pred = md.predict(X_test)
        rscrore=r2_score(y_test,pred)
        if rscrore>accu:
            accu=rscrore
            best_state=i
    return best_state

def best_cv(model,X_train,y_train):
    best_state=0
    accu=0
    for i in range(5,15):
        md = model
        md.fit(X_train,y_train)
        crv=cross_val_score(md,X_train,y_train,cv=i).mean()
        if crv>accu:
            accu=crv
            best_state=i
    if best_state==0:
        return 5
    return best_state


def pipeline_models(X,target):
    parameter_lis=[parameters_linear_reg,parameters_knn_reg,parameters_svr_reg,parameters_deci_tree,parameters_sgd]
    models=[LinearRegression(),KNeighborsRegressor(),SVR(),DecisionTreeRegressor(),SGDRegressor()]
    best_score=0
    best_model_name=''
    for model,parameter in zip(models,parameter_lis):
        rnd_state=best_rn_state(model,X,target)
        print("Best Random State : ",rnd_state)
        X_train, X_test, y_train, y_test = train_test_split(X,target, test_size=0.25,random_state=rnd_state)
        bst_cv=best_cv(model,X_train,y_train)
        print("Best CV is :", bst_cv)
        gsv=GridSearchCV(estimator=model,param_grid=parameter,cv=bst_cv,scoring='r2')
        
        gsv.fit(X_train,y_train)
        print("Traning Score : ",gsv.score(X_train,y_train),"\n\n")
        pred=gsv.best_estimator_.predict(X_test)
        r2score=r2_score(y_test,pred)
        crv=cross_val_score(model,X_train,y_train,cv=bst_cv).mean()
        print(f"Cross Validate score is {(crv*100)}\n\nFinal accuracy is {r2score*100} \n\n{gsv.best_estimator_}\n\n{gsv.best_params_}\n\n=====================================================")
        if r2score>best_score and crv>0:
            best_score=r2score
            best_model_name=[]
            best_model_name.append(model)

    return f"Best model name is :  {best_model_name[0]}"

In [160]:
X=df_new.drop("Price new",axis=1)
target=df_new['Price new']
pipeline_models(X,target)

Best Random State :  195
Best CV is : 9
Traning Score :  0.9983667109838799 


Cross Validate score is 61.65734192671195

Final accuracy is 74.97690967672541 

LinearRegression()

{'copy_X': True, 'fit_intercept': True, 'normalize': False}

Best Random State :  135
Best CV is : 5
Traning Score :  1.0 


Cross Validate score is -11.912584436981554

Final accuracy is 33.50799530215649 

KNeighborsRegressor(n_neighbors=9, weights='distance')

{'n_neighbors': 9, 'weights': 'distance'}

Best Random State :  239
Best CV is : 5
Traning Score :  -0.01029084283539694 


Cross Validate score is -1.7449416305782541

Final accuracy is 0.052589836982464 

SVR(kernel='poly', max_iter=500)

{'gamma': 'scale', 'kernel': 'poly', 'max_iter': 500}

Best Random State :  107
Best CV is : 14
Traning Score :  0.9686184481465142 


Cross Validate score is 56.20805830634203

Final accuracy is 76.64619162007936 

DecisionTreeRegressor(criterion='mae', max_depth=10, max_features='auto')

{'criterion': 'mae', 'ma

'Best model name is :  DecisionTreeRegressor()'

### Cars 24

In [3]:
df1=pd.read_csv(r'C:/data_sci_dec/internship_projects/cars24.csv')
df1.head(3)

,Unnamed: 0,Car_name,Price,Kilometers,Year of Purchase,Owner,Fuel,RTO,Insurance Type,Engine,Interior,Exterior
0,0,2013 Maruti Swift ZXI,"₹ 3,65,299","35,746 km",July 2013,Second Owner,Petrol,MH46,Comp,"Engine Sound, Exhaust Smoke, Clutch, Engine Bl...","Airbags, Music System, AC Cooling, Power windo...","Roof, Bonnet/Hood, Dicky/Boot Door, Alloy whee..."
1,1,2012 Maruti Alto K10 LXI,"₹ 1,79,099","55,072 km",January 2012,Third Owner,Petrol,MH12,Third Party,"Engine Sound, Exhaust Smoke, Clutch, Engine Bl...","Music System, AC Cooling, Heater, Upholstery -...","Roof, Bonnet/Hood, Dicky/Boot Door, Boot Floor"
2,2,2014 Hyundai Xcent SX 1.2,"₹ 4,58,599","23,325 km",May 2014,Second Owner,Petrol,MH01,Comp,"Engine Sound, Exhaust Smoke, Clutch, Engine Bl...","Airbags, Music System, AC Cooling, Power windo...","Roof, Bonnet/Hood, Dicky/Boot Door, Alloy whee..."


In [4]:
df1.shape

(949, 12)

In [5]:
df1['Fuel'].value_counts()

Petrol          655
Diesel          271
Petrol + CNG     22
Petrol + LPG      1
Name: Fuel, dtype: int64

In [6]:
price_new=[]
for i in df1['Price'].str.strip("₹ "):
    price_new.append(int(''.join(i.split(","))))
df1['Price new']=price_new
df1['Price new']

0      365299
1      179099
2      458599
3      309499
4      282199
        ...  
944    780000
945    786199
946    700000
947    650000
948    945000
Name: Price new, Length: 949, dtype: int64

In [7]:
price_new=[]
kms=[]
purchase_yr=[]
for i in df1['Price'].str.strip("₹ "):
    price_new.append(int(''.join(i.split(","))))

fuel=pd.get_dummies(df1['Fuel'])
ins=pd.get_dummies(df1['Insurance Type'])

df1=pd.concat([fuel,ins,df1],axis=1)
for i in df1['Kilometers'].str.strip("km"):
    kms.append(int(''.join(i.split(","))))
df1['Kilometers new']=kms
df1.drop(["Price","Kilometers","Unnamed: 0","Fuel",'Insurance Type'],axis=1,inplace=True)
df1['Owner'].replace(list(df1['Owner'].value_counts().keys()),[1,2,3,4],inplace=True)

for i in df1['Year of Purchase']:
    purchase_yr.append(int(i.split()[1]))
df1['Year of Purchase']=purchase_yr

In [8]:
df1

,Diesel,Petrol,Petrol + CNG,Petrol + LPG,Comp,Insurance Expired,Third Party,Third_party,Zero Depreciation,Zero_Dep,Car_name,Year of Purchase,Owner,RTO,Engine,Interior,Exterior,Price new,Kilometers new
0,0,1,0,0,1,0,0,0,0,0,2013 Maruti Swift ZXI,2013,2,MH46,"Engine Sound, Exhaust Smoke, Clutch, Engine Bl...","Airbags, Music System, AC Cooling, Power windo...","Roof, Bonnet/Hood, Dicky/Boot Door, Alloy whee...",365299,35746
1,0,1,0,0,0,0,1,0,0,0,2012 Maruti Alto K10 LXI,2012,3,MH12,"Engine Sound, Exhaust Smoke, Clutch, Engine Bl...","Music System, AC Cooling, Heater, Upholstery -...","Roof, Bonnet/Hood, Dicky/Boot Door, Boot Floor",179099,55072
2,0,1,0,0,1,0,0,0,0,0,2014 Hyundai Xcent SX 1.2,2014,2,MH01,"Engine Sound, Exhaust Smoke, Clutch, Engine Bl...","Airbags, Music System, AC Cooling, Power windo...","Roof, Bonnet/Hood, Dicky/Boot Door, Alloy whee...",458599,23325
3,0,1,0,0,1,0,0,0,0,0,2016 Maruti Alto 800 LXI,2016,1,MH14,"Engine Sound, Exhaust Smoke, Clutch, Engine Bl...","Music System, AC Cooling, Power windows, Heate...","Roof, Bonnet/Hood, Dicky/Boot Door, Spare Tyre...",309499,18780
4,0,1,0,0,1,0,0,0,0,0,2014 Maruti Alto K10 VXI,2014,1,MH12,"Engine Sound, Exhaust Smoke, Clutch, Engine Bl...","AC Cooling, Power windows, Heater, Upholstery ...","Roof, Bonnet/Hood, Dicky/Boot Door, Boot Floor",282199,29178
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
944,0,1,0,0,0,0,1,0,0,0,2015 Hyundai Elantra SX AT,2015,1,MH04,"Engine Sound, Exhaust Smoke, Clutch, Engine Bl...","Airbags, Music System, AC Cooling, Power windo...",Roof,780000,41053
945,1,0,0,0,1,0,0,0,0,0,2011 BMW X1 SDRIVE 20D,2011,1,MH03,"Exhaust Smoke, Clutch, Engine Blow by Back com...","Airbags, Music System, AC Cooling, Power windo...","Roof, Bonnet/Hood, Dicky/Boot Door",786199,107932
946,1,0,0,0,0,0,1,0,0,0,2017 Mahindra TUV300 T8,2017,1,MH48,"Engine Sound, Exhaust Smoke, Engine Blow by Ba...","Airbags, Music System, AC Cooling, Power windo...",Roof,700000,63816
947,1,0,0,0,0,0,1,0,0,0,2013 Mahindra XUV500 W8 FWD,2013,1,MH43,"Engine Sound, Exhaust Smoke, Clutch, Engine Bl...","Airbags, Music System, AC Cooling, Power windo...",NaN,650000,73574


In [9]:
df1.select_dtypes("object").columns

Index(['Car_name', 'RTO', 'Engine', 'Interior', 'Exterior'], dtype='object')

In [10]:
df1['all_text']=df1['Car_name']+df1['RTO']+df1['Engine']+df1['Interior']+df1['Exterior']

In [11]:
df1.drop(['Car_name', 'RTO', 'Engine', 'Interior', 'Exterior'],axis=1,inplace=True)

In [12]:
df1

,Diesel,Petrol,Petrol + CNG,Petrol + LPG,Comp,Insurance Expired,Third Party,Third_party,Zero Depreciation,Zero_Dep,Year of Purchase,Owner,Price new,Kilometers new,all_text
0,0,1,0,0,1,0,0,0,0,0,2013,2,365299,35746,"2013 Maruti Swift ZXIMH46Engine Sound, Exhaust..."
1,0,1,0,0,0,0,1,0,0,0,2012,3,179099,55072,"2012 Maruti Alto K10 LXIMH12Engine Sound, Exha..."
2,0,1,0,0,1,0,0,0,0,0,2014,2,458599,23325,"2014 Hyundai Xcent SX 1.2MH01Engine Sound, Exh..."
3,0,1,0,0,1,0,0,0,0,0,2016,1,309499,18780,"2016 Maruti Alto 800 LXIMH14Engine Sound, Exha..."
4,0,1,0,0,1,0,0,0,0,0,2014,1,282199,29178,"2014 Maruti Alto K10 VXIMH12Engine Sound, Exha..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
944,0,1,0,0,0,0,1,0,0,0,2015,1,780000,41053,"2015 Hyundai Elantra SX ATMH04Engine Sound, Ex..."
945,1,0,0,0,1,0,0,0,0,0,2011,1,786199,107932,"2011 BMW X1 SDRIVE 20DMH03Exhaust Smoke, Clutc..."
946,1,0,0,0,0,0,1,0,0,0,2017,1,700000,63816,"2017 Mahindra TUV300 T8MH48Engine Sound, Exhau..."
947,1,0,0,0,0,0,1,0,0,0,2013,1,650000,73574,NaN


In [13]:
df1.dropna(inplace=True)

In [23]:
cv=TfidfVectorizer()
text_features=cv.fit_transform(df1['all_text'])

In [24]:
feature_df=pd.DataFrame(text_features.toarray())

In [29]:
df1=pd.concat([df1,feature_df],axis=1).dropna()

In [33]:
df1.drop('all_text',axis=1,inplace=True)

In [34]:
X=df1.drop("Price new",axis=1)
target=df1['Price new']
pipeline_models(X,target)

Best Random State :  0
Best CV is : 5
Traning Score :  0.905346494683455 


Cross Validate score is -8.622663433622072e+16

Final accuracy is -10164774058.546442 

LinearRegression()

{'copy_X': True, 'fit_intercept': True, 'normalize': False}

Best Random State :  0
Best CV is : 5
Traning Score :  0.1458433434725689 


Cross Validate score is -17.540898958140787

Final accuracy is -2.0463828399972073 

KNeighborsRegressor(n_neighbors=9)

{'n_neighbors': 9, 'weights': 'uniform'}

Best Random State :  0
Best CV is : 5
Traning Score :  -0.07178612749254976 


Cross Validate score is -7.732155647981824

Final accuracy is -9.806914946476585 

SVR(kernel='poly', max_iter=500)

{'gamma': 'scale', 'kernel': 'poly', 'max_iter': 500}

Best Random State :  63
Best CV is : 5
Traning Score :  0.6494729097670717 


Cross Validate score is 0.5129556475541808

Final accuracy is 51.806851857793134 

DecisionTreeRegressor(max_depth=5, max_features='auto')

{'criterion': 'mse', 'max_depth': 5, 'max_feat

'Best model name is :  DecisionTreeRegressor()'